In [ ]:
import sys
sys.argv.append('--JOB_NAME')
sys.argv.append('address-cleaning')

sys.argv.append('--source_catalog_database')
sys.argv.append('housing-repairs-raw-zone')

sys.argv.append('--source_catalog_table')
sys.argv.append('housing_repairs_fire_alarmaov')

sys.argv.append('--cleaned_repairs_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-refined-zone/housing-repairs/repairs-electrical-mechanical-fire/fire-alarmaov/cleaned')

In [ ]:
import re
import sys

from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH
from pyspark.sql import types as t
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, trim, when, max, trim
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, TimestampType


def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

def getLatestPartitions(dfa):
   dfa = dfa.where(col('import_year') == dfa.select(max('import_year')).first()[0])
   dfa = dfa.where(col('import_month') == dfa.select(max('import_month')).first()[0])
   dfa = dfa.where(col('import_day') == dfa.select(max('import_day')).first()[0])
   return dfa

In [ ]:
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

source_catalog_database = get_glue_env_var('source_catalog_database', '')
source_catalog_table    = get_glue_env_var('source_catalog_table', '')
cleaned_repairs_s3_bucket_target = get_glue_env_var('cleaned_repairs_s3_bucket_target', '')


sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

logger.info('Fetch Source Data')

source_data = glueContext.create_dynamic_frame.from_catalog(
    name_space=source_catalog_database,
    table_name=source_catalog_table
) 

df = source_data.toDF()
df = getLatestPartitions(df)

In [ ]:
df.printSchema()

In [ ]:
# drop empty rows
df2 = df.filter(df.address != 'nan')

In [ ]:
df2.show(vertical=True)

In [ ]:
logger.info('convert date columns to datetime / date field types')
df2 = df2.withColumn('date', F.to_timestamp('date', 'dd.MM.yy'))

df2.show(vertical=True)

In [ ]:
# drop empty and unused columns
df2 = df2.drop('unnamed:_13', 
               'unnamed:_14',
               'unnamed:_15',
               'unnamed:_16',
               'unnamed:_17',
               'unnamed:_18',
               'unnamed:_19',
               'unnamed:_20',
               'unnamed:_21',
               'unnamed:_22',
               'unnamed:_23',
               'unnamed:_24',
               'unnamed:_25',
               'unnamed:_26',
               'unnamed:_27',
               'unnamed:_28',
               'unnamed:_29',
               'unnamed:_30',
               'unnamed:_31'
              )

In [ ]:
# add new data source column to specify which repairs sheet the repair came from
df2 = df2.withColumn('data_source', F.lit('ElecMechFire - Fire Alarm AOV'))

# rename column names
df2 = df2.withColumnRenamed('address', 'property_address') \
    .withColumnRenamed('description', 'description_of_work') \
    .withColumnRenamed('date', 'datetime_raised') \
    .withColumnRenamed('temp_order_number', 'temp_order_number_full') \
    .withColumnRenamed('priority_code', 'work_priority_description') \
    .withColumnRenamed('cost', 'order_value') \
    .withColumnRenamed('subjective', 'budget_code') \
    .withColumnRenamed('contractor_s_own_ref_no', 'contractor_ref') \
    .withColumnRenamed('contractor_job_status_complete_or_in_progress', 'order_status') \
    .withColumnRenamed('date_completed', 'completed_date') \
    .withColumnRenamed('requested_by', 'operative')

In [ ]:
# remove any spaces from 'work_priority_description' column so that values can be mapped where applicable
df2 = df2.withColumn('work_priority_description', trim(df2.work_priority_description))

def map_repair_priority(code):
    if code == 'Immediate':
        return 1
    elif code == 'Emergency':
        return 2
    elif code == 'Urgent':
        return 3
    elif code == 'Normal':
        return 4
    else:
        return None
    

# # convert to a UDF Function by passing in the function and the return type of function (string in this case)
udf_map_repair_priority = F.udf(map_repair_priority, StringType())
# apply function
df2 = df2.withColumn('work_priority_priority_code', udf_map_repair_priority('work_priority_description'))

In [ ]:
df2.show(vertical=True)

In [ ]:
df2.printSchema()

In [ ]:
cleanedDataframe = DynamicFrame.fromDF(df2, glueContext, "cleanedDataframe")
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=cleanedDataframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": cleaned_repairs_s3_bucket_target,"partitionKeys": ["import_year", "import_month", "import_day", "import_date"]},
    transformation_ctx="parquetData")
job.commit()